In [ ]:
import pandas as pd


# Load all available indicators
df = pd.read_csv('../data/who_obesity_data.csv')
df.shape

In [ ]:
# Examine dimension values
print("Dim1 (Sex) Value counts:")
print(df['Dim1'].value_counts(dropna=False))
print("\n" + "="*50 + "\n")

print("Dim2 (Categories) - Value counts:")
print(df['Dim2'].value_counts(dropna=False))
print("\n" + "="*50 + "\n")

print("Dim1Type values:")
print(df['Dim1Type'].value_counts(dropna=False))
print("\n" + "="*50 + "\n")

print("Dim2Type values:")
print(df['Dim2Type'].value_counts(dropna=False))

In [ ]:

# Check all Dim1Type categories and their top values
print("=" * 60)
print("DIM1 TYPES AND THEIR VALUES")
print("=" * 60)

for dim_type in df['Dim1Type'].dropna().unique():
    dim_values = df[df['Dim1Type'] == dim_type]['Dim1'].value_counts().head(10)
    print(f"\n{dim_type}:")
    print(dim_values)
    print("-" * 40)

print("\n\n")
print("=" * 60)
print("DIM2 TYPES AND THEIR VALUES")
print("=" * 60)

for dim_type in df['Dim2Type'].dropna().unique():
    dim_values = df[df['Dim2Type'] == dim_type]['Dim2'].value_counts().head(10)
    print(f"\n{dim_type}:")
    print(dim_values)
    print("-" * 40)

In [ ]:
# Define aggregate/total values for each dimension type
aggregate_filters = {
    # Dim1 aggregates
    'SEX': ['SEX_BTSX'],  # Both sexes
    'RESIDENCEAREATYPE': ['RESIDENCEAREATYPE_TOTL'],  # Total (urban + rural)
    'AGEGROUP': ['AGEGROUP_YEARSALL'],  # All ages
    'WEALTHQUINTILE': ['WEALTHQUINTILE_TOTL'],  # Total wealth
    'EDUCATIONLEVEL': ['EDUCATIONLEVEL_TOTL'],  # Total education
    'SEVERITY': ['SEVERITY_TOTAL'],  # Total severity
    'HOUSEHOLDWEALTH': ['HOUSEHOLDWEALTH_TOTL'],  # Total wealth
    'WEALTHTERCILE': ['WEALTHTERCILE_TOTL'],  # Total wealth
    'ALCOHOLTYPE': ['ALCOHOLTYPE_SA_TOTAL'],  # Total alcohol
    'TB_TREATMENTTYPE': ['TB_TREATMENTTYPE_TB_TREATMENTTYPE_TOTL'],  # Total TB treatment
    'CONSUMPTIONTYPE': ['CONSUMPTIONTYPE_CONSUMPTION_TOTAL'],  # Total consumption
    'ASSISTIVETECHPRODUCT': ['ASSISTIVETECHPRODUCT_ASSISTIVETECH_TOTAL'],  # Total assistive tech

    # Dim2 aggregates
    'GHECAUSE': ['GHECAUSE_GHE000000'],  # All causes
    'ENVCAUSE': ['ENVCAUSE_ENVCAUSE000'],  # All environmental causes (likely)
}

# Also handle AGEGROUP in Dim2
dim2_agegroup_aggregates = ['AGEGROUP_AGEAll', 'AGEGROUP_YEARS18-PLUS']

df_filtered = df.copy()

# Filter Dim1 to aggregates
for dim_type, allowed_values in aggregate_filters.items():
    df_filtered = df_filtered[
        (df_filtered['Dim1Type'] != dim_type) |  # Keep if not this dimension type
        (df_filtered['Dim1'].isin(allowed_values))  # Or keep if it's an aggregate value
    ]

# Filter Dim2 to aggregates
for dim_type, allowed_values in aggregate_filters.items():
    if dim_type == 'AGEGROUP':
        # Special handling for AGEGROUP in Dim2
        df_filtered = df_filtered[
            (df_filtered['Dim2Type'] != dim_type) |
            (df_filtered['Dim2'].isin(dim2_agegroup_aggregates))
        ]
    else:
        df_filtered = df_filtered[
            (df_filtered['Dim2Type'] != dim_type) |
            (df_filtered['Dim2'].isin(allowed_values))
        ]

# Count countries per indicator
indicator_coverage = df_filtered.groupby(['IndicatorCode',
'IndicatorName'])['SpatialDim'].nunique().reset_index(name='Country_Count')
indicator_coverage = indicator_coverage.sort_values('Country_Count', ascending=False)

print(f"Total records before filtering: {len(df):,}")
print(f"Total records after filtering: {len(df_filtered):,}")
print(f"Reduction: {((len(df) - len(df_filtered)) / len(df) * 100):.1f}%")
print(f"\nNumber of unique indicators: {indicator_coverage.shape[0]}")
print(f"\nTop 200 indicators by country coverage:")


# Remove BMI indicators and add only NCD_BMI_30A indicator
top200Indicators = indicator_coverage[~indicator_coverage['IndicatorCode'].str.lower().str.contains('bmi')].head(200)
top200Indicators = pd.concat([indicator_coverage[indicator_coverage['IndicatorCode'] == 'NCD_BMI_30A'],top200Indicators]).reset_index(drop=True)

with pd.option_context('display.max_rows', None):
    display(top200Indicators)

top200Indicators[['IndicatorCode', 'IndicatorName']].to_csv('../data/metadata/who_indicators_obesity_core.csv',index=False)

In [ ]:
df_core = pd.read_csv('../data/who_obesity_core_data.csv')
df_core.shape

In [ ]:
df_core.head()


In [ ]:
df_core.describe()

In [ ]:
cols = ['IndicatorCode', 'IndicatorName',
    'SpatialDim', # Country code
    'SpatialDimType',  
    'TimeDim',  # Year
    'Dim1Type', 'Dim1',  # (e.g., SEX)
    'Dim2Type', 'Dim2',  # (e.g., AGEGROUP)
    'NumericValue', 'Value',  # The actual data
    'Low', 'High'  # Confidence intervals
]
#filter columns
df_tidy = df_core.copy()[cols]

df_tidy = df_core[[
    'IndicatorCode', 'IndicatorName',
    'SpatialDim',  'TimeDim',
    'Dim1Type', 'Dim1',
    'Dim2Type', 'Dim2',
    'NumericValue', 'Value',
    'Low', 'High'
]].copy()

# 2. Rename for clarity
df_tidy = df_tidy.rename(columns={
'SpatialDim': 'Country',
'TimeDim': 'Year',
'NumericValue': 'Numeric_Value',
'Value': 'Text_Value'
})

# 3. Create a unified value column (choose numeric when available)
df_tidy['Measurement'] = df_tidy['Numeric_Value'].fillna(df_tidy['Text_Value'])

# 4. Separate by Sex if Dim1Type is SEX
df_tidy['Sex'] = df_tidy.apply(
lambda x: x['Dim1'] if x['Dim1Type'] == 'SEX' else 'Both',
axis=1
)

# 5. Separate by Age if Dim2Type is AGEGROUP
df_tidy['AgeGroup'] = df_tidy.apply(
lambda x: x['Dim2'] if x['Dim2Type'] == 'AGEGROUP' else 'All',
axis=1
)

# 6. Drop the generic Dim columns
df_tidy = df_tidy.drop(columns=['Dim1Type', 'Dim1', 'Dim2Type', 'Dim2'])

# 7. Final tidy structure
df_tidy = df_tidy[[
'IndicatorCode', 'IndicatorName',
'Country', 'Year',
'Sex', 'AgeGroup',
'Measurement', 'Low', 'High'
]]





In [ ]:
# Cell: Find indicators with strong correlation to NCD_BMI_30A

# 1. Filter to numeric measurements only
df_analysis = df_tidy[pd.to_numeric(df_tidy['Measurement'],
errors='coerce').notna()].copy()
df_analysis['Measurement'] = pd.to_numeric(df_analysis['Measurement'])

# 2. Filter to consistent dimension levels (Both sexes, All ages)
df_analysis = df_analysis[
    (df_analysis['Sex'].isin(['SEX_BTSX', None])) &
    (df_analysis['AgeGroup'].isin(['AGEGROUP_AGEAll', 'AGEGROUP_YEARS18-PLUS', None]))        
]

# 3. Create unique key for matching (Country + Year)
df_analysis['key'] = df_analysis['Country'] + '_' + df_analysis['Year'].astype(str)

# 4. Pivot: Make indicators as columns, keep country-year as rows
df_pivot = df_analysis.pivot_table(
    index='key',
    columns='IndicatorCode',
    values='Measurement',
    aggfunc='first'  # Use first value if duplicates
)

print(f"Pivot shape: {df_pivot.shape}")
print(f"Number of country-year observations: {df_pivot.shape[0]}")
print(f"Number of indicators: {df_pivot.shape[1]}")

# 5. Check if NCD_BMI_30A exists
if 'NCD_BMI_30A' not in df_pivot.columns:
    print("\n⚠️ NCD_BMI_30A not found in pivoted data!")
    print("Available BMI indicators:", [col for col in df_pivot.columns if 'BMI' in col])     
else:
    # 6. Calculate correlations with NCD_BMI_30A
    # Only calculate for indicators with sufficient overlap
    min_observations = 30  # Minimum shared observations for reliable correlation

    correlations = []
    bmi_data = df_pivot['NCD_BMI_30A']

    for col in df_pivot.columns:
        if col == 'NCD_BMI_30A':
            continue

        # Count valid pairs
        valid_pairs = bmi_data.notna() & df_pivot[col].notna()
        n_pairs = valid_pairs.sum()

        if n_pairs >= min_observations:
            corr = bmi_data.corr(df_pivot[col])
            correlations.append({
                'IndicatorCode': col,
                'Correlation': corr,
                'N_observations': n_pairs
            })

    corr_df = pd.DataFrame(correlations).sort_values('Correlation', ascending=False)

    # 7. Filter for strong correlations (|correlation| > 0.5)
    strong_corr = corr_df[corr_df['Correlation'].abs() > 0.4].copy()

    print(f"\n✅ Found {len(strong_corr)} indicators with |correlation| > 0.4")
    print(f"   (from {len(corr_df)} total indicators with sufficient data)")

    # 8. Add indicator names
    indicator_names = df_tidy[['IndicatorCode', 'IndicatorName']].drop_duplicates()
    strong_corr = strong_corr.merge(indicator_names, on='IndicatorCode', how='left')

    # Reorder columns
    strong_corr = strong_corr[['IndicatorCode', 'IndicatorName', 'Correlation',
'N_observations']]

    print("\n📊 Strongly correlated indicators (sorted by correlation):")
    display(strong_corr)

    # 9. Save results
  #  strong_corr.to_csv('../data/metadata/indicators_correlated_with_obesity.csv',
#index=False)
 #   print(f"\n💾 Saved to: data/metadata/indicators_correlated_with_obesity.csv")

   